# DS-GA 3001.009 Modeling Time Series Data

# Week 5 Hidden Markov Model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import pylab
from collections import Counter
from hmmlearn import hmm
import time

## Part I Data Loading

Load the Wall Street Journal POS dataset. Transform them into indices. 

In [3]:
train_dir = "../../data/en-wsj-train.pos"
test_dir = "../../data/en-wsj-dev.pos"

def load_pos_data(data_dir, word_indexer=None, label_indexer=None, top_k=20000):
    """
    Function that loads the data
    """
    with open(data_dir, "r") as f:
        # load data
        content = f.readlines()
        intermediate_X = []
        intermediate_y = []
        current_X = []
        current_y = []
        vocab_counter = Counter()
        label_set = set()
        for line in content:
            tokens = line.replace("\n", "").replace("$", "").split("\t")
            if len(tokens) <= 1:
                intermediate_X.append(current_X)
                intermediate_y.append(current_y)
                current_X = []
                current_y = []
            elif len(tokens[1]) > 0:
                vocab_counter[tokens[0]]+=1
                label_set.add(tokens[1])
                current_X.append(tokens[0].lower())
                current_y.append(tokens[1])
        # index data
        top_k_words = vocab_counter.most_common(top_k)
        # 0 is reserved for unknown words
        word_indexer = word_indexer if word_indexer is not None else dict([(top_k_words[i][0], i+1) for i in range(len(top_k_words))])
        word_indexer["UNK"] = 0 
        label_indexer = label_indexer if label_indexer is not None else dict([(label, i) for i, label in enumerate(label_set)])
        output_X = []
        output_y = []
        current_X = []
        current_y = []
        for i in range(len(intermediate_X)):
            for j in range(len(intermediate_X[i])):
                if intermediate_X[i][j] in word_indexer:
                    current_X.append(word_indexer[intermediate_X[i][j]])
                else:
                    current_X.append(0)
                current_y.append(label_indexer[intermediate_y[i][j]])
            # populate holders
            output_X.append(current_X)
            output_y.append(current_y)
            # reset current holder
            current_X = []
            current_y = []
        return output_X, output_y, label_indexer, word_indexer, {v: k for k, v in label_indexer.items()}, {v: k for k, v in word_indexer.items()}

In [4]:
train_X, train_y, label_indexer, word_indexer, label_lookup, vocab_lookup = load_pos_data(train_dir)
test_X, test_y, _, _, _, _ = load_pos_data(test_dir, word_indexer=word_indexer, label_indexer=label_indexer)

In [5]:
len(train_X)
len(test_X)

1700

## Part II HMM Implementation

In this part, you will implement the Hidden Markov Model with following methods:


- sample: given a initial state and the number of steps, returns a sequence of sampled states and observations.


- fit: update the transition matrix, emission matrix, and the initial state probability. Note that in our case, the hidden states are given. We don't need to use EM for the learning.


- decode_single_chain: use the Viterbi Algorithm to find the most probable sequence of states.

In [6]:
def reconstruct_sequence(idx_list, lookup):
    """
    Function that reconstructs a sequence of index
    """
    return [lookup[x] for x in idx_list]

def percentage_agree(x, y):
    """
    Function that shows the % of agreement among two list
    """
    assert len(x)==len(y)
    return float(np.sum(np.array(x)==np.array(y)))/len(x)

class MyHMM:
    def __init__(self, num_unique_states, num_observations):
        """
        Constructor
        @param num_unique_states: # of unique states (POS Tags)
        @param num_observations: # of unique observations (words)
        """
        self.num_unique_states = num_unique_states
        self.num_observations = num_observations
        self.transition_matrix = np.zeros((num_unique_states, num_unique_states))
        self.emission_matrix = np.zeros((num_unique_states, num_observations))
        self.initial_states_vector = np.zeros(num_unique_states)
    
    def fit(self, X, y):
        """
        Method that fits the model.
        @param X: array-like with dimension [# of examples, # of length]
        @param y: array-like with dimension [# of examples, # of length]
        """
        # populate holders
        for i in range(len(y)):
            for j in range(len(y[i])):
                # update initial state probability
                if j == 0:
                    self.initial_states_vector[y[i][j]] += 1
                # update transition matrix
                if j < len(y[i])-1:
                    self.transition_matrix[y[i][j], y[i][j+1]] += 1
                # update emission matrix
                self.emission_matrix[y[i][j], X[i][j]] += 1
        # normalization
        self.initial_states_vector += 1e-10
        self.initial_states_vector /= np.sum(self.initial_states_vector)
        
        self.transition_matrix += 1e-10
        row_sums_transition_matrix = np.sum(self.transition_matrix, axis=1)
        self.transition_matrix /= row_sums_transition_matrix[:, np.newaxis]
        
        self.emission_matrix += 1e-10
        row_sums_emission_matrix = np.sum(self.emission_matrix, axis=1)
        self.emission_matrix /= row_sums_emission_matrix[:, np.newaxis]
    
    def decode_single_chain(self, x):
        """
        Auxiliary method that uses Viterbi on single chain
        @param X: array-like with dimension [ # of length]
        @return y: array-like with dimension [# of length]
        """
        # init holders
        y = []
        V = np.zeros( (len(x), self.num_unique_states) )
        best_states = np.zeros( (len(x), self.num_unique_states) )
        # populate the dynamic matrix
        for t in range(len(x)):
            for j in range(self.num_unique_states):
                # v[t,j] = max_{i} v[t-1, i] * transition_matrix[i,j] * emission_matrix[j, x_t]
                if t == 0:
                    prev_V = np.log(self.initial_states_vector)
                else:
                    prev_V = V[t-1, :]
                candidate_list = prev_V + np.log(self.transition_matrix[:,j]) + np.log(self.emission_matrix[j, x[t]])
                best_i = np.argmax(candidate_list)
                
                # populate holders
                V[t,j] = candidate_list[best_i]
                best_states[t,j] = int(best_i)
        # trace back: from T-1 to 1 (zero-indexed), don't need to look at t=0, meaning less        
        for t in range(len(x), 0, -1):
            if t == len(x):
                current_j = int(np.argmax(V[t-1, :]))
                y.append(current_j)
            else:
                current_j = int(best_states[t, current_j])
                y.append(current_j)
        # reverse y
        return list(reversed(y))
        
    def decode(self, X):
        """
        Method that performs the Viterbi the model.
        @param X: array-like with dimension [# of examples, # of length]
        @return y: array-like with dimension [# of examples, # of length]
        """
        return [self.decode_single_chain(sample) for sample in X]
    
    def sample(self, n_step, initial_state):
        """
        Method that given initial state and produces n_step states and observations
        @param n_step: integer
        @param initial_state: an integer indicating the state
        """
        states = []
        observations = []
        curent_state = initial_state
        for t in range(n_step):
            transition_prob = self.transition_matrix[curent_state, :]
            curent_state = np.random.choice(list(range(self.num_unique_states)), p=transition_prob)
            emission_prob = self.emission_matrix[curent_state, :]
            observation = np.random.choice(list(range(self.num_observations)), p=emission_prob)
            states.append(curent_state)
            observations.append(observation)
        return states, observations

Learn an HMM

In [20]:
num_states = len(label_indexer)
num_obs = len(word_indexer)
my_hmm = MyHMM(num_states, num_obs)
my_hmm.fit(train_X, train_y)

Use Viterbi to decode a sequence

In [353]:
i = 5
res = my_hmm.decode_single_chain(test_X[i])
print("data: {0} \n pred: {1} \n label: {2}".format(reconstruct_sequence(test_X[i], vocab_lookup),
                                                    reconstruct_sequence(res, label_lookup),
                                                  reconstruct_sequence(test_y[i], label_lookup)))

data: ['this', 'financing', 'system', 'was', 'created', 'in', 'the', 'new', 'law', 'in', 'order', 'to', 'keep', 'the', 'bailout', 'spending', 'from', 'swelling', 'the', 'budget', 'deficit', '.'] 
 pred: ['DT', 'NN', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', 'TO', 'VB', 'DT', 'NN', 'NN', 'IN', 'VBG', 'DT', 'NN', 'NN', '.'] 
 label: ['DT', 'NN', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NN', 'TO', 'VB', 'DT', 'NN', 'NN', 'IN', 'VBG', 'DT', 'NN', 'NN', '.']


In [348]:
start_time = time.time()
pred_train = my_hmm.decode(train_X[:1000])
print("takes {0} seconds".format(time.time() - start_time))

takes 9.493995189666748 seconds


In [354]:
start_time = time.time()
pred_test = my_hmm.decode(test_X)
print("takes {0} seconds".format(time.time() - start_time))

takes 15.467236280441284 seconds


In [350]:
np.mean([percentage_agree(pred_train[i], train_y[i]) for i in range(len(pred_train))])

0.93240962302301245

In [355]:
np.mean([percentage_agree(pred_test[i], test_y[i]) for i in range(len(pred_test))])

0.91945955881678232

Sample

In [352]:
pos_tag, words = my_hmm.sample(10, label_indexer["NNP"])
print(reconstruct_sequence(pos_tag, label_lookup))
print(reconstruct_sequence(words, vocab_lookup))

['NNP', 'CC', 'IN', 'CD', 'NNP', 'VBD', 'VBN', '.', "''", 'VBZ']
['golden', 'and', 'as', '1973', 'UNK', 'rose', 'cut', '.', "''", 'UNK']


### Please turn in the code before 03/01/2018 6:45 pm. Please name your notebook netid.ipynb.

### Your work will be evaluated based on the code and plots. You don't need to write down your answers to these questions in the text blocks. 

### Reference



In [371]:
import sys

for line in sys.stdin:
    print(line)

In [408]:
def haha(my_int):
    max_iter = int(np.floor((np.sqrt(my_int))))
    factor = []
    for i in range(1,max_iter+1):
        if my_int % i == 0:
            factor.append(i)
            if my_int/i != my_int:
                factor.append(my_int/i)
    return len(set(factor))

In [19]:
my_hmm.initial_states_vector

array([  7.03252543e-04,   1.38138892e-03,   3.11440412e-03,
         3.51626272e-04,   5.68881075e-02,   7.28368705e-04,
         4.16677132e-02,   7.54238352e-02,   2.51161623e-15,
         2.03440914e-03,   2.93859098e-03,   2.68742936e-03,
         3.76742434e-04,   6.25392440e-03,   8.53949517e-04,
         3.01393947e-04,   2.51161623e-15,   5.90229813e-03,
         1.22064549e-02,   1.75813136e-04,   2.88835866e-03,
         1.63255055e-03,   2.51161623e-15,   2.51161623e-15,
         4.21951526e-03,   7.78601030e-04,   1.26108251e-01,
         4.01858596e-04,   2.51161623e-05,   2.51161623e-15,
         2.00929298e-04,   2.56184855e-03,   5.83699611e-02,
         4.12909707e-02,   2.00904182e-01,   6.27904056e-04,
         3.61672736e-03,   4.06128344e-02,   2.13487379e-03,
         1.45673741e-03,   6.89689815e-02,   1.22818033e-02,
         2.16928293e-01,   2.51161623e-15])

In [376]:
50%1

0

In [ ]:
np.sqrt()